# Investigating Crop Quality in Beaune, France with Remote Sensing instruments for the 2022 vintage.

The goal of this project is to investigate the tools freely available to the public in monitoring the health of *Vitis Vinifera* grapes in the vineyard. This study will focus on the Burgundy region as it produces some of my favorite wines using Pinot Noir.

Here I investigate Landsat 9 imagery by interfacing with [Google Earth Engine](https://earthengine.google.com/).
I will calculate NDVI and a handful of other vegitation indexes to see which are useful and compare the freely available products to private options which may be better suited.

Particular inspiration for this analysis came from [this study](https://www.frontiersin.org/articles/10.3389/fpls.2021.683078/full) conducted on a vineyard in Greece which concluded that *"both satellite-based and proximal-based NDVI at both stages (veraison and harvest) presented good correlations to crop quality characteristics ..."*. While this seems promising to my preliminary research it should be noted that the proximal based (in this study's case, drones within the vineyards) remote sensing applications were preffered to the remote sensing based (in this case Sentinel-2 archived imagery) instruments though I only have access to the remote sensing instruments and it is what I wish to investigate in this project.
There are private companies like [VineView](https://vineview.com/data-products/vine-vigor-products/) and [VineScapes](https://www.vinescapes.com/read-our-published-work/) already using remote sensing techniques to offer insight into vineyard management.

### The Project's Main Components

1. Visualization of NDVI using Landsat and Sentinel imagery.
2. Analysis of correlation between the region's NDVI and wine rating.


## Visualization
#### The workflow

1.      Retrieve the Landsat-9 Image Collection from Google Earth Engine.
2.      Filter the geographic bounds to our study area around Beaune.
3.      Sort the Image Collection by cloud cover.
4.      Select the first element of the collection (image with least clouds within our timeframe).
5.      Clip resulting image to study area.



In [1]:
# Uncomment these if you do not have the dependencies installed

# !pip install ee
# !pip install geemap
# !pip install pandas

import ee
import geemap

# Get our geemaps map set up
burgundy_map = geemap.Map(center = ( 47.02882844813948, 4.948176654205355), zoom = 11, layer_ctrl = True, data_ctrl = False, add_google_map = False)

Let's grab our data. We'll start with Landsat-9 and move on to Sentinel-2

In [2]:
clip_area = ee.Geometry.Polygon([[[4.647364477,46.9062950917],[5.1135967769,46.9062950917],[5.1135967769,47.1193160686],[4.647364477,47.1193160686],[4.647364477,46.9062950917]]])
study_area = ee.Geometry.Point([4.744932, 47.040085])
study_date = ('2022-08-01', '2022-08-31')


# Get Landsat 9 imagery
image_2022 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterDate(study_date[0], study_date[1]).filterBounds(study_area).sort('CLOUD_COVER').first().clip(clip_area)

Let's investigate our photo's metadata to see what we've got after all that filtering and clipping.

In [3]:
props = geemap.image_props(image_2022)
landsat_date = props.get("IMAGE_DATE").getInfo()
landsat_cloud = (props.get("CLOUD_COVER_LAND").getInfo())

print(f'The clearest Landsat-9 photo during the 2022 harvest season for our region was taken on \033[1m{landsat_date}\033[0m with a cloud cover of \033[1m{landsat_cloud}%\033[0m')

The clearest Landsat-9 photo during the 2022 harvest season for our region was taken on 2022-08-07 with a cloud cover of 0.02%


August 7th, 2022 should work out well for our analysis since the 2022 harvest in Burgundy began around [August 31st](https://www.burgundy-report.com/category/harvests/vintage-2022/). So we will get a look at the vines just a couple weeks before harvest! And on a day with very low cloud coverage!

The weeks prior to harvest are some of the most stressful for a winemaker. Too little or too much of rain, humidity, sunlight, or soil nutrients can dramatically impact the final quality of the wine by veering the grape from the desired PH and sugar levels. While this may not be a problem for more "modern" regions, Burgundy maintains that the role of the vintner in the final product should be minimal and it is their job to simply see through what the terroir has given them.<br>This approach to minimal intervention winemaking makes for more terroir driven wines but leaves the product at the mercy of nature.

Having a snapshot into these stressful times should be incredibly interesting!

### Landsat 9 Data Scaling

All Landsat data needs to be rescaled before any analysis or visualization can occus, you can read more on the topic [here](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2#bands).

In [4]:
def applyScaleFactors(image: ee.Image) -> ee.Image:
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# landsat_9_data_2022 = applyScaleFactors(landsat_9_image_2022)
landsat_8_data_2022 = applyScaleFactors(image_2022)

Let's save the Landsat visualization parameters for our different layers in dicts so we can visualize them more programatically.

In [5]:
landsat_nat = {
    'name': 'Natural Color (4,3,2)',
    'properties': {
        'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
        'min': 0.0,
        'max': 0.3,
    }
}

landsat_IR = {
    'name': 'False Color (5,4,3)',
    'properties': {
        'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
        'min': 0.0,
        'max': 0.5
    }
}

landsat_veg = {
    'name': 'Vegatation Visualization (6,5,4)',
    'properties': {
        'bands': ['SR_B6', 'SR_B5', 'SR_B4'],
        'min': 0.0,
        'max': 0.7
    }
}

vis_ndvi = {
    'name': 'NDVI',
    'properties': {
        'min': -0.2,
        'max': 1.0,
        'palette': [
            'FFFFFF',
            'CE7E45',
            'DF923D',
            'F1B555',
            'FCD163',
            '99B718',
            '74A901',
            '66A000',
            '529400',
            '3E8601',
            '207401',
            '056201',
            '004C00',
            '023B01',
            '012E01',
            '011D01',
            '011301',
        ]
    }
}

# Color scheme "borrowed" from https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/ndvi/
ndvi_legend = {
    'keys' : ['NDVI < -0.2', '-.2 < NDVI ≤ 0', '0 < NDVI ≤ .1', '.1 < NDVI ≤ .2', '.2 < NDVI ≤ .3', '.3 < NDVI ≤ .4', '.4 < NDVI ≤ .5', '.5 < NDVI ≤ .6', '.6 < NDVI ≤ .7', '.7 < NDVI ≤ .8', '.8 < NDVI ≤ .9']
}

### Initial Visualization 

Alright that was a lot.

Let's put that all on a map so we can see what we've been doing so far.<br>We'll put a natural color and an infrared layer to more easily visualize the greenery of the region. We'll call on our previously assigned visualization parameters to tell geemap how to draw the layers.

Feel free to play around with the layers using the tool on the right hand side of the map.

In [6]:
burgundy_map.add_ee_layer(
    ee_object = landsat_8_data_2022,
    vis_params = landsat_IR['properties'],
    name = f'2022 {landsat_IR["name"]}'
)
burgundy_map.add_ee_layer(
    ee_object = landsat_8_data_2022,
    vis_params = landsat_nat['properties'],
    name = f'2022 {landsat_nat["name"]}'
)

burgundy_map

Map(center=[47.02882844813948, 4.948176654205355], controls=(ZoomControl(options=['position', 'zoom_in_text', …

### That looks pretty good!

With Landsat 9's 30m resolution we can clearly see the city of Beaune with the sprawling vineyards and forests to the east.
This will be the first harvest that Landsat 9 imagery can be used for analysis since its launch on September 27 2021.

In our false color layer, we can see the large red patch to the right of the image is the [Forêt domaniale de Citeaux](https://www.onf.fr/vivre-la-foret/forets-de-france/%2B/19b6::foret-domaniale-de-citeaux.html?lang=fr) which is an oak forest producing staves for many of the oak barrels used for aging wines throughout the region. This is why you can see the plot like structure of the forest in both of our layers. Interestingly, only in the false color composite you can distinguish which plots are more mature, and likely to be harvested next whereas in the natural color you can only distinguish larger segmentation of the forest.<br>Try flicking between the layers to see! A fun example of the practicality of remote sensing.

If you need spatial reference, you can unselect all our added layers to see an Open Street Map base layer.

### NDVI Calculation & Visualization

Ndvi has [long been used](https://spinoff.nasa.gov/spinoff2003/er_2.html) in vineyards to generalize vine health.

Farmers use it to monitor crops and decide when is the optimal time to pick the fruit.

The calculation for NDVI is fairly straight forward and involves finding the difference between the near-infrared and red bands of our image.
$$
    NDVI = \frac{(NIR - Red)}{(NIR + Red)}\
$$

Let's calculate the NDVI of our Landsat-9 image using the appropriate bands.

In [7]:
landsat_NIR = image_2022.select('SR_B5')
landsat_red = image_2022.select('SR_B4')

# This is the line that computes NDVI
landsat_ndvi = landsat_NIR.subtract(landsat_red).divide(landsat_NIR.add(landsat_red)).rename('Landsat NDVI')

Let's visualize our NDVI layer.<br>All previous layers are still available to play around with.

In [8]:
burgundy_map.add_ee_layer(
    ee_object = landsat_ndvi,
    name = 'Landsat NDVI',
    vis_params = vis_ndvi['properties']
)
burgundy_map.add_colorbar_branca(
    colors = vis_ndvi['properties']['palette'],
    vmin = vis_ndvi['properties']['min'],
    vmax = vis_ndvi['properties']['max'],
    layer_name = 'Landsat NDVI',
)
burgundy_map

Map(center=[47.02882844813948, 4.948176654205355], controls=(ZoomControl(options=['position', 'zoom_in_text', …

Awesome, we've got our NDVI, and visualization layers overlapping to get a better understanding of the region of Beaune at harvest time.

### What about Sentinel Imagery?

That's all very interesting and the Landsat project is awesome however, [this study](https://www.mdpi.com/2077-0472/11/5/457#) found that *"In medium-resolution imagery, Sentinel-2 excels in use over others, due to the ability to reveal spatial variability that can be used on a regional scale."*.<br>
So let's see what the same NDVI calculation will give us with Sentinel-2 data.<br>
I will repeat much of the same workflow as for the Landsat image.

In [9]:
sentinel_2022 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
    .filterDate(study_date[0], study_date[1])\
    .filterBounds(study_area)\
    .sort('HIGH_PROBA_CLOUDS_PERCENTAGE')\
    .first()\
    .clip(clip_area)

# Have to divide sentinel bands by 10000, similar to Landsat scalling.
# Note this is only for bands 1-12 and breaks all others but that is all we will use for this so a blanket divide works fine.
sentinel_2022_data = sentinel_2022.divide(10000)

Again let's see what the best photo of the same 2022 harvest period was to come out of the Copernicus Sentinel-2 mission.

In [10]:
props2 = geemap.image_props(sentinel_2022)
sentinel_date = props2.get("IMAGE_DATE").getInfo()
sentinel_cloud = round(props2.get("CLOUDY_PIXEL_PERCENTAGE").getInfo(), 2)

print(f'The clearest Sentinel photo during the 2022 harvest season for our region was taken on \033[1m{sentinel_date}\033[0m with a cloud cover of \033[1m{sentinel_cloud}%\033[0m')

The clearest Sentinel photo during the 2022 harvest season for our region was taken on 2022-08-10 with a cloud cover of 0.0%


Wow, the best Sentinel photo for the study time period was just 3 days after our Landsat one with no clouds!<br>

Let's visualize it.

Sentinel uses different band combinations and different syntax for the name of their bands, so we'll create new visualization parameters.

In [11]:
sentinel_nat = {
    'name': 'Natural Color (4,3,2)',
    'properties': {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0.0,
        'max': 0.3,
    }
}

sentinel_IR = {
    'name': 'False Color (8,4,3)',
    'properties': {
        'bands': ['B8', 'B4', 'B3'],
        'min': 0.0,
        'max': 0.5,
    }
}


In [12]:
sentinel_map = geemap.Map(center = ( 47.02882844813948, 4.948176654205355), zoom = 11, layer_ctrl = True, data_ctrl = False, add_google_map = False)

sentinel_map.add_ee_layer(
    ee_object = sentinel_2022_data,
    vis_params = sentinel_nat['properties'],
    name = sentinel_nat['name'],
)

sentinel_map.add_ee_layer(
    ee_object = sentinel_2022_data,
    vis_params = sentinel_IR['properties'],
    name = sentinel_IR['name']
)

sentinel_map

Map(center=[47.02882844813948, 4.948176654205355], controls=(ZoomControl(options=['position', 'zoom_in_text', …

I mean that is awesome isn't it?! Sentinel-2 offers 10m resolution on the bands we used for visualizing our layers and while it is not going to get us into individual rows of vines it sure looks great.

Now let's do that NDVI calculation again being careful to select the correct Sentinel-2 bands for the job.

In [13]:
sentinel_NIR = sentinel_2022_data.select('B8')
sentinel_red = sentinel_2022_data.select('B4')

sentinel_ndvi = sentinel_NIR.subtract(sentinel_red).divide(sentinel_NIR.add(sentinel_red)).rename('Sentinel NDVI')

### Let's get a side by side of our NDVI's using both satellites.

While this split function doesn't work fully as intended and is a [known bug](https://github.com/gee-community/geemap/issues/1483), dragging the map around still has the intended effect and is visually striking so we'll use it regardless.

Remember we are using the same visualization parameters for both images 

In [14]:
# Create tile layers for split map
landsat_layer = geemap.ee_tile_layer(
    landsat_ndvi, vis_params=vis_ndvi['properties'], name='Landsat NDVI')
sentinel_layer = geemap.ee_tile_layer(
    sentinel_ndvi, vis_params=vis_ndvi['properties'], name='Sentinel NDVI')

# Initialize split map
split = geemap.Map(center=(47.02882844813948, 4.948176654205355),
                   zoom=11, layer_ctrl=True, data_ctrl=False, add_google_map=False)

# Add data to aplit map  
split.split_map(left_layer = landsat_layer,
                right_layer = sentinel_layer,
                left_label = f"<center>Landsat 9 NDVI<br>{landsat_date}</center>",
                right_label = f"<center>Sentinel-2 NDVI<br>{sentinel_date}</center>",)
split.add_colorbar_branca(
    colors = vis_ndvi['properties']['palette'],
    vmin = vis_ndvi['properties']['min'],
    vmax = vis_ndvi['properties']['max'],
)


In [15]:
# Visualize it
split

Map(center=[47.02882844813948, 4.948176654205355], controls=(ZoomControl(options=['position', 'zoom_in_text', …

This concludes the visualization portion of the project.

## Correlation of NDVI to Vintage Scores

My next curiosity is to see how the NDVI changes year over year.<br>Vintages in the wine world are the year in which the wine was grown and are incredibly deterministic to the wine's quality thanks to the notion of [terroir](https://winefolly.com/tips/terroir-definition-for-wine/).

Thanks to our preliminary investigation into the freely available remote sensing tools, we will continue this analysis using Sentinel-2 imagery.

#### The workflow


1.      Declare a list of years to iterate over (Sentinel-2 only went up in 2015 so that's our starting point).
2.      Declare an empty dictionary to catch the best image for the harvest season of each year.
3.      Fetch the Sentinel-2 Image Collection.
4.      Filter the date of the Image Collection by the year being iterated over.
5.      Filter the geographic bounds to our study area around Beaune.
6.      Sort the Collection by lowest cloud coverage.
7.      Select the first element of the collection (image with least clouds within our timeframe).
8.      Clip resulting image to study area.
9.      Calculate NDVI on the image.
10.      Add our NDVI image to our map for visualization.
11.      Update our Dictionary with the structure {'year' : NDVI image}.

Unfortunately, most of our *legitimate* analysis will have to end here. We do not have enough of a sample size to realistically conclude a corelation between NDVI and wine scores in the region since Google Earth Engine only has Sentinel-2 imagery from 2017 and the vintage ratings for Beaune only go up to 2020 (2021 is still in barrels).

While this 3 year window is not enough to properly analyze, it sure is enough to armchair analyze.

Let's throw some reviews on to each NDVI layer and you can decide for yourself whether you think NDVI affects the vintage scores.

In [16]:
from tqdm.autonotebook import tqdm

years = [2017, 2018, 2019, 2020, 2021, 2022]
images = {}
ndvi_map = geemap.Map( center = (47.02882844813948, 4.948176654205355) , zoom = 11, layer_ctrl = True, add_google_map = False)

for year in tqdm(years):

    # Get image
    image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
        .filterDate(f'{str(year)}{study_date[0][4:]}', f'{str(year)}{study_date[1][4:]}')\
        .filterBounds(study_area)\
        .sort('HIGH_PROBA_CLOUDS_PERCENTAGE')\
        .first()\
        .clip(clip_area)\

    image_data = image.divide(10000)

    # Calculate NDVI
    NIR = image_data.select('B8')
    red = image_data.select('B4')
    ndvi = NIR.subtract(red).divide(NIR.add(red))

    # Calculate the mean NDVI over the entire region
    mean_ndvi = ndvi.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=clip_area,
        scale=10,
        maxPixels=1e9
    )
    mean_final = mean_ndvi.get("B8").getInfo()

    # Add the NDVI as a layer to our NDVI map
    image_info = geemap.image_props(image)
    date = image_info.get("IMAGE_DATE").getInfo()
    ndvi_map.add_ee_layer(
        ee_object = ndvi,
        vis_params = vis_ndvi['properties'],
        name = f'{date} - Mean NDVI: {round(mean_final, 2)}'
    )
    
    # Let's make sure cloud coverage is acceptable
    # Note this is a value of the entire image, not just our clipped region so in reality it's not too bad
    print(f'Cloud coverage for the best photo from August {year} was {round(image_info.get("CLOUD_COVERAGE_ASSESSMENT").getInfo(), 2)}%')


    # Save the data in the dictionary
    images[str(year)] = {'mean ndvi': mean_final,
                        'image': ndvi}


  0%|          | 0/6 [00:00<?, ?it/s]

Cloud coverage for the best photo from August 2017 was 5.67%
Cloud coverage for the best photo from August 2018 was 21.99%
Cloud coverage for the best photo from August 2019 was 1.66%
Cloud coverage for the best photo from August 2020 was 0.03%
Cloud coverage for the best photo from August 2021 was 5.12%
Cloud coverage for the best photo from August 2022 was 0.0%


Whew that was a lot of processing, luckily Google Earth Engine handles it all [server side](https://www.sciencedirect.com/science/article/pii/S0034425717302900) (free of charge for academic purposes!) so our machines aren't at risk here.

Let's see all the work that was done. Feel free to play around again with the layers to see how NDVI changed in the region from 2017 - 2022.<br>Most striking to me is the difference between 2020 and 2021, try turning all other layers off and flicking between them. One problem is our acceptable time range for a good photo is quite large 

In [17]:
ndvi_map

Map(center=[47.02882844813948, 4.948176654205355], controls=(WidgetControl(options=['position', 'transparent_b…